In [10]:
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# ====== STEP 1: Load the Datasets ======
file_paths = {
    "air_pollution": r"D:\TY\기특대 논문\Nature Climate Change\Code\Regression\data\Air_pollution_impact_factors_KR.xlsx",
    "floods": r"D:\TY\기특대 논문\Nature Climate Change\Code\Regression\data\Floods_impact_factors_KR.xlsx",
    "heat_wave": r"D:\TY\기특대 논문\Nature Climate Change\Code\Regression\data\Heat_wave_impact_factors_KR.xlsx",
    "hurricanes": r"D:\TY\기특대 논문\Nature Climate Change\Code\Regression\data\Hurricanes_impact_factors_KR.xlsx",
    "tweets": r"D:\TY\기특대 논문\Nature Climate Change\Code\Regression\data\Tweet_counts_All_2020_KR.xlsx",
}


# Read each Excel file
dataframes = {key: pd.ExcelFile(path) for key, path in file_paths.items()}

# Load the first sheet from each file
df_air_pollution = dataframes["air_pollution"].parse("Sheet1")
df_floods = dataframes["floods"].parse("Sheet1")
df_heat_wave = dataframes["heat_wave"].parse("Sheet1")
df_hurricanes = dataframes["hurricanes"].parse("Sheet1")
df_tweets = dataframes["tweets"].parse("Sheet1")

# Convert 'Date' columns to datetime format
for df in [df_air_pollution, df_floods, df_heat_wave, df_hurricanes, df_tweets]:
    df["Date"] = pd.to_datetime(df["Date"])


# Rename df_floods columns before merging
df_floods = df_floods.rename(columns={col: col + "_floods" for col in df_floods.columns if col != "Date"})

# Rename df_heat_wave columns before merging
df_heat_wave = df_heat_wave.rename(columns={col: col + "_heat_wave" for col in df_heat_wave.columns if col != "Date"})

# Rename df_hurricanes columns before merging
df_hurricanes = df_hurricanes.rename(columns={col: col + "_hurricanes" for col in df_hurricanes.columns if col != "Date"})

# ====== STEP 2: Merge All Datasets on 'Date' ======
merged_df = df_tweets.merge(df_air_pollution, on="Date", suffixes=("_tweets", "_air_pollution"))
merged_df = merged_df.merge(df_floods, on="Date")  # No need for suffixes since renamed
merged_df = merged_df.merge(df_heat_wave, on="Date")  # No need for suffixes since renamed
merged_df = merged_df.merge(df_hurricanes, on="Date")

# Drop duplicate columns
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

# Print available columns before regression
print("\n✅ Available columns in merged dataset:\n", merged_df.columns.tolist())


# Handle missing values (fill NaNs with 0 assuming no impact)
merged_df.fillna(0, inplace=True)

print("\n✅ Data successfully merged and missing values handled!\n")


# ====== STEP 3: Normalize Impact Factors Safely ======
impact_factor_columns = [col for col in merged_df.columns if any(x in col for x in ["air_pollution", "floods", "heat_wave", "df_hurricanes"])]

# Apply Min-Max Normalization: (X - min) / (max - min)
for col in impact_factor_columns:
    min_val, max_val = merged_df[col].min(), merged_df[col].max()
    if max_val - min_val == 0:
        merged_df[col] = 0  # Avoid division by zero
    else:
        merged_df[col] = (merged_df[col] - min_val) / (max_val - min_val)

print("\n✅ Min-Max Normalization applied to impact factors.\n")


# ====== STEP 4: Automatically Extract All State Names ======
states_to_analyze = [col.replace("_tweets", "") for col in merged_df.columns if "_tweets" in col]



# ====== STEP 5: Multiple Linear Regression & Store Results ======
regression_results_all_states = []

# Run regression for each state
for state in states_to_analyze:
    try:
        print(f"\n📊 Running Regression for {state}...")

        # Dynamically check if columns exist before running regression
        event_columns = [col for col in merged_df.columns if col.startswith(state) and any(x in col for x in ["air_pollution", "floods", "heat_wave", "hurricanes"])]

        if not event_columns:
            print(f"⚠ Skipping {state} due to missing event data.")
            continue

        # Define independent variables (event factors)
        X = merged_df[event_columns]

        # Define dependent variable (tweet counts)
        y = merged_df[f"{state}_tweets"]

        # Drop rows with NaNs or Infs in independent variables
        X.replace([np.inf, -np.inf], np.nan, inplace=True)
        y.replace([np.inf, -np.inf], np.nan, inplace=True)
        X.dropna(inplace=True)
        y.dropna(inplace=True)

        # Skip if no valid data after cleaning
        if X.empty or y.empty:
            print(f"⚠ Skipping {state} due to insufficient valid data.")
            continue

        # Add constant term
        X = sm.add_constant(X)

        # Fit multiple linear regression model
        model = sm.OLS(y, X).fit()

        # Store results in dictionary
        results_dict = {
            "State": state,
            "R-squared": model.rsquared,
            "Adj. R-squared": model.rsquared_adj,
            "F-statistic (p-value)": model.f_pvalue,
            "Air Pollution Coeff": model.params.get(f"{state}_air_pollution", "N/A"),
            "Air Pollution p-value": model.pvalues.get(f"{state}_air_pollution", "N/A"),
            "Floods Coeff": model.params.get(f"{state}_floods", "N/A"),
            "Floods p-value": model.pvalues.get(f"{state}_floods", "N/A"),
            "Heat Wave Coeff": model.params.get(f"{state}_heat_wave", "N/A"),
            "Heat Wave p-value": model.pvalues.get(f"{state}_heat_wave", "N/A"),
            "Hurricanes Coeff": model.params.get(f"{state}_hurricanes", "N/A"),
            "Hurricanes p-value": model.pvalues.get(f"{state}_hurricanes", "N/A")
        }

        # # Store coefficients and p-values for each event factor
        # for col in event_columns:
        #     results_dict[f"{col} Coeff"] = model.params.get(col, "N/A")
        #     results_dict[f"{col} p-value"] = model.pvalues.get(col, "N/A")

        regression_results_all_states.append(results_dict)

    except KeyError as e:
        print(f"Skipping {state} due to missing data: {e}")
    except Exception as e:
        print(f"Unexpected error for {state}: {e}")

# Convert results to DataFrame
df_regression_results_all_states = pd.DataFrame(regression_results_all_states)




# ====== STEP 6: Save Results to Excel ======
df_regression_results_all_states.to_excel(r"D:\TY\기특대 논문\Nature Climate Change\Code\Regression\result\regression_results_norm_KR.xlsx", index=False)

print("\n✅ Regression analysis completed! Results saved as 'regression_results_norm_KR.xlsx'.\n")

# Print the final results table
print(df_regression_results_all_states)


✅ Available columns in merged dataset:
 ['Date', '서울_tweets', '부산_tweets', '대구_tweets', '인천_tweets', '광주_tweets', '대전_tweets', '울산_tweets', '세종_tweets', '경기_tweets', '강원_tweets', '충남_tweets', '충북_tweets', '전남_tweets', '전북_tweets', '경남_tweets', '경북_tweets', '제주_tweets', '서울_air_pollution', '부산_air_pollution', '대구_air_pollution', '인천_air_pollution', '광주_air_pollution', '대전_air_pollution', '울산_air_pollution', '세종_air_pollution', '경기_air_pollution', '강원_air_pollution', '충남_air_pollution', '충북_air_pollution', '전남_air_pollution', '전북_air_pollution', '경남_air_pollution', '경북_air_pollution', '제주_air_pollution', '서울_floods', '부산_floods', '대구_floods', '인천_floods', '광주_floods', '대전_floods', '울산_floods', '세종_floods', '경기_floods', '강원_floods', '충남_floods', '충북_floods', '전남_floods', '전북_floods', '경남_floods', '경북_floods', '제주_floods', '서울_heat_wave', '부산_heat_wave', '대구_heat_wave', '인천_heat_wave', '광주_heat_wave', '대전_heat_wave', '울산_heat_wave', '세종_heat_wave', '경기_heat_wave', '강원_heat_wave', '충남_heat

C:\Users\taeyong0209\AppData\Local\Temp\ipykernel_28916\3295673226.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\taeyong0209\AppData\Local\Temp\ipykernel_28916\3295673226.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.dropna(inplace=True)
C:\Users\taeyong0209\AppData\Local\Temp\ipykernel_28916\3295673226.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.